In [ ]:
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import numpy as np

import ramanchada2 as rc2
import ramanchada2.misc.constants as rc2const
import ramanchada2.misc.utils as rc2utils
from scipy import interpolate

In [ ]:
def calc_pos_ampl(spe):
    cand = spe.find_peak_multipeak(prominence=spe.y_noise*2, sharpening=None)
    pos, ampl = np.array([[k, v] for k, v in cand.get_pos_ampl_dict().items()]).T
    return pos, ampl

def calc_pos(spe):
    ret, _ = calc_pos_ampl(spe)
    return ret

In [ ]:
def argmin2d(A):
    ymin_idx = np.argmin(A, axis=0)
    xmin_idx = np.argmin(A, axis=1)
    x_idx = np.unique(xmin_idx[xmin_idx[ymin_idx[xmin_idx]] == xmin_idx])
    y_idx = np.unique(ymin_idx[ymin_idx[xmin_idx[ymin_idx]] == ymin_idx])
    matches = np.stack([y_idx, x_idx]).T
    return matches


def find_closest_pairs_idx(x, y):
    outer_dif = np.abs(np.subtract.outer(x, y))
    return argmin2d(outer_dif).T

In [ ]:
# load experimental spectrum
spe = rc2.spectrum.from_test_spe(laser_wl=['785'], sample=['Neon'], OP=['03']).normalize()
pos, ampl = calc_pos_ampl(spe)

In [ ]:
# load reference data
ref_pos, ref_ampl = np.array([[k, v] for k, v in rc2const.neon_rs_785_nist_dict.items()]).T

In [ ]:
pos_idx, ref_pos_idx = find_closest_pairs_idx(pos, ref_pos)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.matshow(np.abs(np.subtract.outer(pos, ref_pos)).T)
ax.plot(pos_idx, ref_pos_idx, 'ro')

# Polyharmonic spline

In [ ]:
a = interpolate.RBFInterpolator(pos[pos_idx].reshape(-1, 1), ref_pos[ref_pos_idx])

In [ ]:
cal_spe_polyh = spe.__copy__()
cal_spe_polyh.x = a(spe.x.reshape(-1, 1))

In [ ]:
fig, ax = plt.subplots(nrows=2, figsize=(15, 8))
ax[0].stem(ref_pos, ref_ampl/np.max(ref_ampl), linefmt='-r', markerfmt='.', basefmt='k', label='Reference')
ax[0].plot(spe.x, spe.y, label='original spectrum')
ax[0].plot(cal_spe_polyh.x, spe.y, label='polyharmonic calibrated')
ax[0].legend()
ax[0].grid()

ax[1].stem(ref_pos, ref_ampl/np.max(ref_ampl), linefmt='-r', markerfmt='.', basefmt='k', label='Reference')
ax[1].plot(spe.x, spe.y, label='original spectrum')
ax[1].plot(cal_spe_polyh.x, spe.y, label='polyharmonic calibrated')
ax[1].legend()
ax[1].grid()
ax[1].set_xlim(800, 1500)
fig.tight_layout()

In [ ]:
fig, ax = plt.subplots()
ax.plot(spe.x, cal_spe_polyh.x - spe.x, ',')
ax.set_title('$x_{original} - x_{calibrated; polyharmonic}$')
None

# Fine calib GG with iterations

In [ ]:
cal_spe = spe
fig, ax = plt.subplots()

for _ in range(10):
    ax.plot(spe.x, cal_spe.x - spe.x)
    cal_spe = cal_spe.xcal_fine(ref=list(rc2const.neon_rs_785_nist_dict.keys()), poly_order=3)

ax.set_title('$x_{original} - x_{calibrated; iterations}$')
None

In [ ]:
fig, ax = plt.subplots(nrows=2, figsize=(15, 8))
ax[0].stem(ref_pos, ref_ampl/np.max(ref_ampl), linefmt='-r', markerfmt='.', basefmt='k', label='Reference')
ax[0].plot(spe.x, spe.y, label='original spectrum')
ax[0].plot(cal_spe.x, spe.y, label='iter calibrated')
ax[0].legend()
ax[0].grid()

ax[1].stem(ref_pos, ref_ampl/np.max(ref_ampl), linefmt='-r', markerfmt='.', basefmt='k', label='Reference')
ax[1].plot(spe.x, spe.y, label='original spectrum')
ax[1].plot(cal_spe.x, spe.y, label='iter calibrated')
ax[1].legend()
ax[1].grid()
ax[1].set_xlim(800, 1500)
fig.tight_layout()

In [ ]:
def plot(iter):
    fig, ax = plt.subplots(figsize=(20, 14
                                   ), tight_layout=True)
    ax.set_title(f'Iteration {iter}')
    ref_pos, ref_ampl = np.array([[k, v] for k, v in rc2const.neon_rs_785_nist_dict.items()]).T

    ax.eventplot(np.array([pos]).T, 'horizontal', linelengths=100, lineoffsets=pos, linewidths=.5, colors='r', linestyles=':')
    ax.eventplot(np.array([ref_pos]).T, 'vertical', linelengths=100, lineoffsets=ref_pos, linewidths=.5)
    twinx = ax.twinx()
    twinx.plot(spe.x, spe.y, 'r:', label="original spe")
    twinx.set_ylim(.005, 2000)
    twinx.set_yscale('log')
    twiny = ax.twiny()
    twinx.stem(ref_pos, ref_ampl/np.max(ref_ampl)*np.max(spe.y), linefmt='-', markerfmt='', basefmt='k', label='reference NIST')

    twiny.stem(ref_pos, ref_ampl, orientation='horizontal', linefmt='-', markerfmt='', basefmt='k')
    twiny.set_xlim(xmax=150)

    cal_spe = spe
    for _ in range(iter):
        cal_spe = cal_spe.xcal_fine(ref=list(rc2const.neon_rs_785_nist_dict.keys()), poly_order=3)
    twinx.plot(cal_spe.x, cal_spe.y, label='calibrated spe')
    ax.plot([100, 2500], [100, 2500], ':', color='cyan', lw=.5)
    
    cal_pos = calc_pos(cal_spe)
    pos_idx, ref_pos_idx = rc2utils.find_closest_pairs_idx(cal_pos, ref_pos)
    pos_match, ref_pos_match = cal_pos[pos_idx], ref_pos[ref_pos_idx]
    ax.plot(pos_match, ref_pos_match, '1')
    
    subax = ax.inset_axes([.51, .1, .48, .4])
    #subax.stem(ref_pos, ref_ampl/np.max(ref_ampl), linefmt='-', markerfmt='.', basefmt='k', label='Ref')
    #subax.set_yticks([])
    #subax.set_xlim(600, 1500)
    #subax.set_xlim(0, 600)
    #subax.plot(cal_spe.x, cal_spe.y/np.max(cal_spe.y), label='Exp')
    #subax.legend()
    subax.plot(cal_spe.x, cal_spe.x-spe.x)
    subax.set_title('$x_{cal}-x_{orig}$')
    #twinx.set_yticks([])
    #twiny.set_xticks([])
    
    subax_dif = ax.inset_axes([.12, .68, .48, .27])
    subax_dif.plot(ref_pos_match, ref_pos_match-pos_match)
    subax_dif.set_title('reference - experimental')
    subax_dif.grid()
    
    ax.set_xlabel('Experimental data [$\mathrm{cm}^{-1}$]')
    ax.set_ylabel('NIST reference data [$\mathrm{cm}^{-1}$]')
    twinx.legend()

In [ ]:
with PdfPages('xcal-fine.pdf') as pdf:
    for i in range(10):
        plot(i)
        pdf.savefig()